## Project Week 1 Review:
___ 
 [1] Compiled all the individual script files into an intractive jupyter notebook  
 [2] Done with the singificant and necessary code changes in order to run with latest libraries   
 [3] Added the necessary code to train the model using `DenseNet` which is SOTA on imagenet Dataset.  
 ___

In [ ]:
# Install the required libraries
!pip install Keras==2.1.6 opencv-python==3.4.1.15 scipy==1.1.0 tensorflow-gpu==1.8.0 Theano==1.0.2

In [1]:
import pickle
import os
import numpy as np
import sys
import time
import cv2
import random

In [3]:
# To supress the unnecessary warnings  
import warnings
warnings.filterwarnings('ignore')

# --- If you are using this jupyter notebook behind the proxy network ---
# os.environ['http_proxy']="http://proxy-server:proxy-port"
# os.environ['https_proxy']="http://proxy-server:proxy-port"

#### Download the BoxCar116k dataset 
___

In [4]:
!wget https://medusa.fit.vutbr.cz/traffic/data/BoxCars116k.zip
!unzip BoxCars116k.zip #update the BOXCARS_DATASET_ROOT from the below cell before submission

#### Utitlity Funcitons :
___

In [5]:
def load_cache(path, encoding="latin-1", fix_imports=True):
    """
    encoding latin-1 is default for Python2 compatibility
    """
    with open(path, "rb") as f:
        return pickle.load(f, encoding=encoding, fix_imports=True)

#%%
def save_cache(path, data):
    with open(path, "wb") as f:
        pickle.dump(data, f)

#%%
def ensure_dir(d):
    if len(d)  == 0: # for empty dirs (for compatibility with os.path.dirname("xxx.yy"))
        return
    if not os.path.exists(d):
        try:
            os.makedirs(d)
        except OSError as e:
            if e.errno != 17: # FILE EXISTS
                raise e
                
#%%
def download_report_hook(block_num, block_size, total_size):
    downloaded = block_num*block_size
    percents = downloaded / total_size * 100
    show_str = " %.1f%%"%(percents)
    sys.stdout.write(show_str + len(show_str)*"\b")
    sys.stdout.flush()
    if downloaded >= total_size:
        print()


#### Download the pre-trained model inorder to reproduce the numerical results of [BoxCars: Improving Vehicle Fine-Grained Recognition using 3D Bounding Boxes in Traffic Surveillance](https://arxiv.org/pdf/1703.00686.pdf)
___
Net | Original 3DBBs | Estimated 3DBBs | Image Processing Time
----|---------------:|---------------:|---------------------:
ResNet50 |  84.29/91.61 | 81.78/90.79  | 5.8ms
VGG16 | 84.10/92.09 | 81.43/90.68 | 5.4ms
VGG19 | 83.35/91.23 | 81.93/91.48  | 5.4ms
InceptionV3 | 81.51/89.86 | 79.89/89.92 | 6.1ms


In [26]:
# Parameters to by filled by the user
#%%
MODELS_DIR_URL = "https://medusa.fit.vutbr.cz/traffic/data/BoxCars-models/"
SUFFIX = "h5"

#output directory where to put downloaded models
DEFAULT_OUTPUT_DIR = os.path.realpath(os.path.join(".", "models")) 

all_nets = True
net_name = None #download all available models 

In [29]:
# -*- coding: utf-8 -*-
import urllib.request 
import re

# --- If you are using this jupyter notebook behind the proxy network
# handler=urllib.request.ProxyHandler({'http':'http://proxy-server:proxy-port',
#                                      'https':'http://proxy-server:proxy-port'})
# opener=urllib.request.build_opener(handler)
# urllib.request.install_opener(opener)

#%%
with urllib.request.urlopen(MODELS_DIR_URL) as response:
    dir_listing = response.read().decode("utf-8")

model_matcher = re.compile(r'href="(.*)\.%s"'%(SUFFIX))
available_nets = model_matcher.findall(dir_listing)

print("Download trained model files. Available nets: %s"%(str(available_nets)))

download_nets = net_name
if all_nets:
    download_nets = available_nets
    
if len(download_nets) == 0:
    print("You need to specify net_name to download or use all_nets=True to download all of them\nAVAILABLE NETS: %s\n"%(str(available_nets)))
    sys.exit(1)

#%%
print("Saving downloaded models to: %s"%(DEFAULT_OUTPUT_DIR))
ensure_dir(DEFAULT_OUTPUT_DIR)
for net in download_nets:
    if net not in available_nets:
        print("WARNING: Skipping %s because it is not available. AVAILABLE_NETS: %s"%(net, str(available_nets)))
        continue
    print("Downloading %s... \n"%(net), end="")
    sys.stdout.flush()
    urllib.request.urlretrieve(MODELS_DIR_URL + net + "." + SUFFIX, os.path.join(args.output_dir, "%s.%s"%(net, SUFFIX)), download_report_hook)
    


Download trained model files. Available nets: ['InceptionV3', 'InceptionV3_estimated3DBB', 'ResNet50', 'ResNet50_estimated3DBB', 'VGG16', 'VGG16_estimated3DBB', 'VGG19', 'VGG19_estimated3DBB']
Saving downloaded models to: /project/BoxCars_new/scripts/models

#### Configure the necessary directories:
___

In [6]:
#%%
# change this to your location
# BOXCARS_DATASET_ROOT = "/mnt/matylda1/isochor/Datasets/BoxCars116k/" 
BOXCARS_DATASET_ROOT = "BoxCars116k/"
#%%
BOXCARS_IMAGES_ROOT = os.path.join(BOXCARS_DATASET_ROOT, "images")
BOXCARS_DATASET = os.path.join(BOXCARS_DATASET_ROOT, "dataset.pkl")
BOXCARS_ATLAS = os.path.join(BOXCARS_DATASET_ROOT, "atlas.pkl")
BOXCARS_CLASSIFICATION_SPLITS = os.path.join(BOXCARS_DATASET_ROOT, "classification_splits.pkl")

#### BoxDataset Class Object :
___

In [7]:
#%%
class BoxCarsDataset(object):
    def __init__(self, load_atlas = False, load_split = None, use_estimated_3DBB = False, estimated_3DBB_path = None):
        self.dataset = load_cache(BOXCARS_DATASET)
        self.use_estimated_3DBB = use_estimated_3DBB
        
        self.atlas = None
        self.split = None
        self.split_name = None
        self.estimated_3DBB = None
        self.X = {}
        self.Y = {}
        for part in ("train", "validation", "test"):
            self.X[part] = None
            self.Y[part] = None # for labels as array of 0-1 flags
            
        if load_atlas:
            self.load_atlas()
        if load_split is not None:
            self.load_classification_split(load_split)
        if self.use_estimated_3DBB:
            self.estimated_3DBB = load_cache(estimated_3DBB_path)
        
    #%%
    def load_atlas(self):
        self.atlas = load_cache(BOXCARS_ATLAS)
    
    #%%
    def load_classification_split(self, split_name):
        self.split = load_cache(BOXCARS_CLASSIFICATION_SPLITS)[split_name]
        self.split_name = split_name
       
    #%%
    def get_image(self, vehicle_id, instance_id):
        """
        returns decoded image from atlas in RGB channel order
        """
        return cv2.cvtColor(cv2.imdecode(self.atlas[vehicle_id][instance_id], 1), cv2.COLOR_BGR2RGB)
        
    #%%
    def get_vehicle_instance_data(self, vehicle_id, instance_id, original_image_coordinates=False):
        """
        original_image_coordinates: the 3DBB coordinates are in the original image space
                                    to convert them into cropped image space, it is necessary to subtract instance["3DBB_offset"]
                                    which is done if this parameter is False. 
        """
        vehicle = self.dataset["samples"][vehicle_id]
        instance = vehicle["instances"][instance_id]
        if not self.use_estimated_3DBB:
            bb3d = self.dataset["samples"][vehicle_id]["instances"][instance_id]["3DBB"]
        else:
            bb3d = self.estimated_3DBB[vehicle_id][instance_id]
            
        if not original_image_coordinates:
            bb3d = bb3d - instance["3DBB_offset"]

        return vehicle, instance, bb3d 
            
       
    #%%
    def initialize_data(self, part):
        assert self.split is not None, "load classification split first"
        assert part in self.X, "unknown part -- use: train, validation, test"
        assert self.X[part] is None, "part %s was already initialized"%part
        data = self.split[part]
        x, y = [], []
        for vehicle_id, label in data:
            num_instances = len(self.dataset["samples"][vehicle_id]["instances"])
            x.extend([(vehicle_id, instance_id) for instance_id in range(num_instances)])
            y.extend([label]*num_instances)
        self.X[part] = np.asarray(x,dtype=int)

        y = np.asarray(y,dtype=int)
        y_categorical = np.zeros((y.shape[0], self.get_number_of_classes()))
        y_categorical[np.arange(y.shape[0]), y] = 1
        self.Y[part] = y_categorical
        print ("initialize_data\n", self.X[part].shape, self.Y[part].shape)
        


    def get_number_of_classes(self):
        return len(self.split["types_mapping"])
        
        
    def evaluate(self, probabilities, part="test", top_k=1):
        samples = self.X[part]
        print (samples.shape, probabilities.shape)
        assert samples.shape[0] == probabilities.shape[0]
        assert self.get_number_of_classes() == probabilities.shape[1]
        part_data = self.split[part]
        probs_inds = {}
        for vehicle_id, _ in part_data:
            probs_inds[vehicle_id] = np.zeros(len(self.dataset["samples"][vehicle_id]["instances"]), dtype=int)
        for i, (vehicle_id, instance_id) in enumerate(samples):
            probs_inds[vehicle_id][instance_id] = i
            
        get_hit = lambda probs, gt: int(gt in np.argsort(probs.flatten())[-top_k:])
        hits = []
        hits_tracks = []
        for vehicle_id, label in part_data:
            inds = probs_inds[vehicle_id]
            hits_tracks.append(get_hit(np.mean(probabilities[inds, :], axis=0), label))
            for ind in inds:
                hits.append(get_hit(probabilities[ind, :], label))
                
        return np.mean(hits), np.mean(hits_tracks)


#### Boxcars Image Transformations Auxilary Methods :
___

In [8]:
# -*- coding: utf-8 -*-

#%%
def alter_HSV(img, change_probability = 0.6):
    if random.random() < 1-change_probability:
        return img
    addToHue = random.randint(0,179)
    addToSaturation = random.gauss(60, 20)
    addToValue = random.randint(-50,50)
    hsvVersion =  cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    channels = hsvVersion.transpose(2, 0, 1)
    channels[0] = ((channels[0].astype(int) + addToHue)%180).astype(np.uint8)
    channels[1] = (np.maximum(0, np.minimum(255, (channels[1].astype(int) + addToSaturation)))).astype(np.uint8)
    channels[2] = (np.maximum(0, np.minimum(255, (channels[2].astype(int) + addToValue)))).astype(np.uint8)
    hsvVersion = channels.transpose(1,2,0)   
        
    return cv2.cvtColor(hsvVersion, cv2.COLOR_HSV2RGB)

#%%
def image_drop(img, change_probability = 0.6):
    if random.random() < 1-change_probability:
        return img
    width = random.randint(int(img.shape[1]*0.10), int(img.shape[1]*0.3))
    height = random.randint(int(img.shape[0]*0.10), int(img.shape[0]*0.3))
    x = random.randint(int(img.shape[1]*0.10), img.shape[1]-width-int(img.shape[1]*0.10))
    y = random.randint(int(img.shape[0]*0.10), img.shape[0]-height-int(img.shape[0]*0.10))
    img[y:y+height,x:x+width,:] = (np.random.rand(height,width,3)*255).astype(np.uint8)
    return img

#%%
def add_bb_noise_flip(image, bb3d, flip, bb_noise):
    bb3d = bb3d + bb_noise 
    if flip:
        bb3d[:, 0] = image.shape[1] - bb3d[:,0]
        image = cv2.flip(image, 1)
    return image, bb3d

#%%
def _unpack_side(img, origPoints, targetSize):
    origPoints = np.array(origPoints).reshape(-1,1,2)
    targetPoints = np.array([(0,0), (targetSize[0],0), (0, targetSize[1]), 
                             (targetSize[0], targetSize[1])]).reshape(-1,1,2).astype(origPoints.dtype)
    m, _ = cv2.findHomography(origPoints, targetPoints, 0)
    resultImage = cv2.warpPerspective(img, m, targetSize)
    return resultImage
    
    
#%%    
def unpack_3DBB(img, bb):
    frontal = _unpack_side(img, [bb[0], bb[1], bb[4], bb[5]], (75,124))
    side = _unpack_side(img, [bb[1], bb[2], bb[5], bb[6]], (149,124))
    roof = _unpack_side(img, [bb[0], bb[3], bb[1], bb[2]], (149,100))
    
    final = np.zeros((224,224,3), dtype=frontal.dtype)
    final[100:, 0:75] = frontal
    final[0:100, 75:] = roof
    final[100:, 75:] = side
    
    return final
    

#### BoxDataGenerator Object to populate the data in batches : 
___

In [9]:
from keras.preprocessing.image import Iterator

#%%
class BoxCarsDataGenerator(Iterator):
    def __init__(self, dataset, part, batch_size=8, training_mode=False, seed=None, generate_y = True, image_size = (224,224)):
        assert image_size == (224,224), "only images 224x224 are supported by unpack_3DBB for now, if necessary it can be changed"
        assert dataset.X[part] is not None, "load some classification split first"
        super().__init__(dataset.X[part].shape[0], batch_size, training_mode, seed)
        self.part = part
        self.generate_y = generate_y
        self.dataset = dataset
        self.image_size = image_size
        self.training_mode = training_mode
        if self.dataset.atlas is None:
            self.dataset.load_atlas()
            
    #%%
    def _get_batches_of_transformed_samples(self, index_array):
        current_batch_size = len(index_array)
        x = np.empty([current_batch_size] + list(self.image_size) + [3], dtype=np.float32)
        for i, ind in enumerate(index_array):
            vehicle_id, instance_id = self.dataset.X[self.part][ind]
            vehicle, instance, bb3d = self.dataset.get_vehicle_instance_data(vehicle_id, instance_id)
            image = self.dataset.get_image(vehicle_id, instance_id)
            if self.training_mode:
                image = alter_HSV(image) # randomly alternate color
                image = image_drop(image) # randomly remove part of the image
                bb_noise = np.clip(np.random.randn(2) * 1.5, -5, 5) # generate random bounding box movement
                flip = bool(random.getrandbits(1)) # random flip
                image, bb3d = add_bb_noise_flip(image, bb3d, flip, bb_noise) 
            image = unpack_3DBB(image, bb3d) 
            image = (image.astype(np.float32) - 116)/128.
            x[i, ...] = image
        if not self.generate_y:
            return x
        y = self.dataset.Y[self.part][index_array]
        print (x.shape, y.shape)
        return x, y
        
        

Using TensorFlow backend.


#### Trainging Related Auxilary Functions:
___

In [10]:
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.densenet import DenseNet169,DenseNet121,DenseNet201
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Flatten, Dropout, AveragePooling2D
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard

Training Related External Arguments:
___

In [11]:
estimated_3DBB_path = None  #use estimated 3DBBs from specified path
evaluation_path = 'models/ResNet50.h5' #path to model file to be evaluated
resume_path = None  #path to model file to be resumed

output_final_model_path = None  
tensorboard_dir = None
snapshots_dir = None

training_network = "ResNet50" # train on one of following available nets: ["ResNet50", "VGG16", "VGG19", "InceptionV3", "DenseNet121", "DenseNet169", "DenseNet201"]
cache_path = "../cache/"  #where to store training meta-data and final model
'''------------------------------------------------------------'''
lr = 0.0025 #learning rate
batch_size = 16
epochs = 10 #run for epochs

In [12]:
#%% initialize dataset object
if estimated_3DBB_path is None:
    dataset = BoxCarsDataset(load_split="hard", load_atlas=True)
else:
    dataset = BoxCarsDataset(load_split="hard", load_atlas=True, 
                             use_estimated_3DBB = True, estimated_3DBB_path = estimated_3DBB_path)

In [13]:
#%% get optional path to load model
model = None
for path in [evaluation_path, resume_path]:
    if path is not None:
        print("Loading model from %s"%path)
        model = load_model(path)
        break

Loading model from ../models/ResNet50.h5


In [14]:
if model is None:
    print("Initializing new %s model ..."%training_network)
    if training_network in ("ResNet50", ):
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
        x = Flatten()(base_model.output)
    
    if training_network in ("DenseNet121", "DenseNet169", "DenseNet201"):
        if training_network == "DenseNet121":
            base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))
        elif training_network == "DenseNet169":
            base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(224,224,3))
        elif training_network == "DenseNet201":
            base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224,224,3))
        x = Flatten()(base_model.output)
        
    if training_network in ("VGG16", "VGG19"):
        if training_network == "VGG16":
            base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
        elif training_network == "VGG19":
            base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
        x = Flatten()(base_model.output)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dropout(0.5)(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dropout(0.5)(x)

    if training_network in ("InceptionV3", ):
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))
        output_dim = int(base_model.outputs[0].get_shape()[1])
        x = AveragePooling2D((output_dim, output_dim), strides=(output_dim, output_dim), name='avg_pool')(base_model.output)
        x = Flatten()(x)
    
            
    predictions = Dense(dataset.get_number_of_classes(), activation='softmax')(x)
    model = Model(input=base_model.input, output=predictions, name="%s%s"%(training_network, {True: "_estimated3DBB", False:""}[estimated_3DBB_path is not None]))
    optimizer = SGD(lr=lr, decay=1e-4, nesterov=True)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    #model.summary()

In [15]:
print("Model name: %s"%(model.name))
if estimated_3DBB_path is not None and "estimated3DBB" not in model.name:
    print("ERROR: using estimated 3DBBs with model trained on original 3DBBs")
    sys.exit(1)
if estimated_3DBB_path is None and "estimated3DBB" in model.name:
    print("ERROR: using model trained on estimated 3DBBs and running on original 3DBBs")
    sys.exit(1)

output_final_model_path = os.path.join(cache_path, model.name, "final_model.h5")
snapshots_dir = os.path.join(cache_path, model.name, "snapshots")
tensorboard_dir = os.path.join(cache_path, model.name, "tensorboard")

Model name: ResNet50


In [16]:
#%% training
if evaluation_path is None:
    print("Training...")
    #%% initialize dataset for training
    dataset.initialize_data("train")
    dataset.initialize_data("validation")
    generator_train = BoxCarsDataGenerator(dataset, "train", batch_size, training_mode=True)
    generator_val = BoxCarsDataGenerator(dataset, "validation", batch_size, training_mode=False)


    #%% callbacks
    ensure_dir(tensorboard_dir)
    ensure_dir(snapshots_dir)
    tb_callback = TensorBoard(tensorboard_dir, histogram_freq=1, write_graph=False, write_images=False)
    saver_callback = ModelCheckpoint(os.path.join(snapshots_dir, "model_{epoch:03d}_{val_acc:.2f}.h5"), period=4 )

    #%% get initial epoch
    initial_epoch = 0
    if resume_path is not None:
        initial_epoch = int(os.path.basename(resume_path).split("_")[1]) + 1


    model.fit_generator(generator=generator_train, 
                        steps_per_epoch=generator_train.n,
                        epochs=epochs,
                        verbose=1,
                        validation_data=generator_val,
                        validation_steps=generator_val.n,
                        callbacks=[tb_callback, saver_callback],
                        initial_epoch = initial_epoch,
                        )

    #%% save trained data
    print("Saving the final model to %s"%(output_final_model_path))
    ensure_dir(os.path.dirname(output_final_model_path))
    model.save(output_final_model_path)

#### The model is evaluated when the training is finished or can be evaluated on saved weights
___

In [ ]:
#%% evaluate the model 
print("Running evaluation...")
dataset.initialize_data("test")
generator_test = BoxCarsDataGenerator(dataset, "test", batch_size, training_mode=False, generate_y=False)
start_time = time.time()
predictions = model.predict_generator(generator_test, generator_test.n, verbose=1)
end_time = time.time()
single_acc, tracks_acc = dataset.evaluate(predictions)
print(" -- Accuracy: %.2f%%"%(single_acc*100))
print(" -- Track accuracy: %.2f%%"%(tracks_acc*100))
print(" -- Image processing time: %.1fms"%((end_time-start_time)/generator_test.n*1000))

Running evaluation...
initialize_data
 (39149, 2) (39149, 107)
10589/39149 [=======>......................] - ETA: 26:12